In [4]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
#upload crash data file
df= pd.read_csv('/content/drive/MyDrive/Data_Washington Fatal Crash Survey.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (21,22,23,301,303) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
# define function to retrieve zip code for given coordinates
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='my_app')

def get_zipcode(lat, lon):
    try:
        location = geolocator.reverse(f'{lat}, {lon}')
        address = location.raw['address']
        return address.get('postcode')
    except (AttributeError, GeocoderTimedOut):
        return None

# define function to retrieve zip codes in parallel using multiprocessing
def get_zipcodes_parallel(row):
    return (row['y'], row['x'], get_zipcode(row['y'], row['x']))


In [ ]:
#apply function to each row of the dataframe in parallel to retrieve postal code for each accident
from geopy.exc import GeocoderTimedOut
from multiprocessing import Pool, cpu_count
with Pool(processes=cpu_count()) as pool:
    results = pool.map(get_zipcodes_parallel, df.to_dict('records'))

# update dataframe with postal codes
df['zip_code'] = [result[2] for result in results]
# can save updated dataframe to a new CSV file or proceed to clustering
#df.to_csv('Data_Washington_zipcodes.csv', index=False)

In [6]:
#import zipcode data
zipcenters= pd.read_csv('/content/drive/MyDrive/WAzips.csv')
#this is the whole usa
half= zipcenters[zipcenters['ZIP'] > 98000]
wazipcenters= half[half['ZIP'] < 99403]

In [7]:
#prep as numpy array for clustering
WZC=pd.DataFrame(wazipcenters['LAT']) 
WZC['LNG']= wazipcenters['LNG']
z= np.array(WZC)
z

array([[  47.310617, -122.263291],
       [  47.308286, -122.216812],
       [  47.304801, -122.316969],
       ...,
       [  46.800013, -118.311494],
       [  46.087207, -117.249856],
       [  46.190773, -117.127417]])

In [8]:
#read crash data or continue from above
#fatalcrash = pd.read_csv('/content/drive/MyDrive/Data_Washington_zipcodes.csv')
fatalcrash=df
fatalcrash.head()

,year,case,par,repjur,crash_dt,crash_tm,accday,accmon,holiday,county,...,nmdistract6,lab,isedtcase,crf1,crf2,crf3,CoRoadName,CoMP,IntCoRoadName,IntCoMP
0,2017,1,E628946,2.0,01/01/2017,2:12,1,1,1.0,61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,2,E627989,26.0,01/02/2017,17:14,2,1,1.0,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,4,3747633,263.0,01/01/2017,18:47,1,1,1.0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,5,E628691,4.0,01/01/2017,3:50,1,1,1.0,63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,6,3746306,263.0,01/05/2017,9:53,5,1,0.0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#prep fpr clustering
FC=pd.DataFrame() 
FC['LAT']=fatalcrash['y']
FC['LNG']= fatalcrash['x']
crashcord=np.array(FC)
crashcord



array([[  48.02354722, -122.1756167 ],
       [  47.43134167, -122.583225  ],
       [  47.6887    , -122.3364222 ],
       ...,
       [  47.58708611, -122.3048944 ],
       [  47.58708611, -122.3048944 ],
       [  47.11046944, -123.0058917 ]])

In [10]:
#loop all the Ks to cluster crash data and get score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
testingk={}
X = crashcord 
w=len(WZC)
l=np.arange(2,w)

for i in l:
  kmeans = KMeans(n_clusters=i, init='k-means++', n_init=1, random_state= 98101).fit(X)
  #results=pd.Series(kmeans.labels_)
  #FC[i]=results
  testingk[i]=silhouette_score(X, kmeans.labels_)
sorted(testingk.items(), key=lambda x:x[1])

[(54, 0.38181101508058535),
 (48, 0.38584947946534603),
 (49, 0.38720943868366703),
 (46, 0.38741124390516485),
 (53, 0.3877223313635704),
 (47, 0.3883311956014509),
 (50, 0.3902366551086745),
 (51, 0.39624122245601207),
 (52, 0.39778735696852424),
 (42, 0.3990802098859634),
 (43, 0.3997613061903037),
 (45, 0.40067638634035124),
 (41, 0.40245302521641646),
 (44, 0.40419792775057395),
 (112, 0.4112366233401974),
 (113, 0.41222216796126765),
 (102, 0.41240672370586934),
 (103, 0.4124644323088773),
 (108, 0.4124836321520174),
 (111, 0.41259523802672093),
 (40, 0.4128529054629337),
 (110, 0.4129711862643877),
 (109, 0.4131161801773557),
 (114, 0.4131179671920488),
 (115, 0.41380685589904825),
 (116, 0.41451524597434375),
 (101, 0.4147759383771725),
 (104, 0.4154951011602196),
 (107, 0.416036198483623),
 (105, 0.41679381191134507),
 (119, 0.41685404539049226),
 (94, 0.416944179639836),
 (117, 0.4170255177859185),
 (118, 0.41721571385096984),
 (106, 0.4172555075628092),
 (95, 0.4172964140650

In [ ]:
#loop all the Ks and get score for clustered zipcodes
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
testingk={}
X = crashcord 
w=len(WZC)
l=np.arange(2,w)

for i in l:
  kmeans = KMeans(n_clusters=i, init='k-means++', n_init=1, random_state= 98101).fit(z)
  #results=pd.Series(kmeans.labels_)
  #FC[i]=results
  testingk[i]=silhouette_score(z, kmeans.labels_)
sorted(testingk.items(), key=lambda x:x[1])

[(596, 0.0012098510645193567),
 (594, 0.0024886349593584854),
 (595, 0.0026910441608847427),
 (593, 0.005942481746305151),
 (592, 0.008348917010954957),
 (591, 0.010127724154050178),
 (590, 0.013096902407192431),
 (589, 0.015035361623159991),
 (588, 0.015950551525200394),
 (587, 0.01741150117207437),
 (586, 0.018991045026205047),
 (585, 0.019654924881034055),
 (584, 0.021339125371290336),
 (583, 0.02267096257465966),
 (582, 0.023191272818426558),
 (581, 0.025038897819610488),
 (579, 0.02540688937213519),
 (580, 0.026885157484817297),
 (578, 0.027843192049096962),
 (577, 0.029187905287646605),
 (576, 0.029847066415071153),
 (575, 0.031244631501301245),
 (574, 0.03148625780922973),
 (573, 0.033555382170471146),
 (572, 0.034227380977657325),
 (571, 0.03447195451020079),
 (570, 0.03629148976644473),
 (569, 0.0364937589170427),
 (568, 0.037183267719869705),
 (567, 0.03855262589510125),
 (562, 0.039301197304363156),
 (565, 0.03930666415426453),
 (561, 0.03952202149963051),
 (564, 0.039551854

In [9]:
#loop all the Ks based on sillouhette score and make dataframes
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
outputk={}
X = crashcord 
scoresk={}
#w=len(WZC)
#l=[8,10,20,50,100,150,200, 300]
l=[10,100,300]
for i in l:
  kmeans = KMeans(n_clusters=i, init='k-means++', n_init=1, random_state= 98101).fit(X)
  results=pd.Series(kmeans.labels_)
  cn="cluster{n}".format(n=i)
  fatalcrash[cn]=results
  scoresk[i]=silhouette_score(X, kmeans.labels_)
  p=kmeans.predict(z)
  cn2="cluster{n}KEY".format(n=i)
  wazipcenters[cn2]=p

<ipython-input-9-ddbb5a8f9d81>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wazipcenters[cn2]=p
<ipython-input-9-ddbb5a8f9d81>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wazipcenters[cn2]=p
<ipython-input-9-ddbb5a8f9d81>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [ ]:
#or process data with just one K
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
outputk={}
X = crashcord 
scoresk={}
#choose K
k=100
kmeans = KMeans(n_clusters=k, init='k-means++', n_init=1, random_state= 98101).fit(X)
results=pd.Series(kmeans.labels_)
cn="cluster{n}".format(n=k)
fatalcrash[cn]=results
scoresk[i]=silhouette_score(X, kmeans.labels_)
p=kmeans.predict(z)
cn2="cluster{n}KEY".format(n=k)
wazipcenters[cn2]=p

In [10]:
#check silouhette scores
scoresk

{10: 0.5075352043445912, 100: 0.4212275873573746, 300: 0.45335739153970467}

In [11]:
#Check labled data
fatalcrash

,year,case,par,repjur,crash_dt,crash_tm,accday,accmon,holiday,county,...,crf2,crf3,CoRoadName,CoMP,IntCoRoadName,IntCoMP,zip_code,cluster10,cluster100,cluster300
0,2017,1,E628946,2.0,01/01/2017,2:12,1,1,1.0,61,...,NaN,NaN,NaN,NaN,NaN,NaN,98201.0,6,7,7
1,2017,2,E627989,26.0,01/02/2017,17:14,2,1,1.0,35,...,NaN,NaN,NaN,NaN,NaN,NaN,98359.0,4,28,129
2,2017,4,3747633,263.0,01/01/2017,18:47,1,1,1.0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,98103.0,4,15,15
3,2017,5,E628691,4.0,01/01/2017,3:50,1,1,1.0,63,...,NaN,NaN,NaN,NaN,NaN,NaN,99219.0,2,66,72
4,2017,6,3746306,263.0,01/05/2017,9:53,5,1,0.0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,98174.0,4,56,149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4127,2021,636,Incid08,334.0,08/08/2021,9:38,8,8,NaN,47,...,NaN,NaN,NaN,NaN,NaN,NaN,99155.0,9,45,78
4128,2021,638,Incid09,334.0,09/05/2021,1:36,5,9,NaN,19,...,NaN,NaN,NaN,NaN,NaN,NaN,3971.0,1,1,1
4129,2021,639,EB48605,263.0,04/21/2021,17:32,21,4,NaN,33,...,NaN,NaN,NaN,NaN,NaN,NaN,98144.0,4,56,216
4130,2021,639,EB48605,263.0,04/21/2021,17:32,21,4,NaN,33,...,NaN,NaN,NaN,NaN,NaN,NaN,98144.0,4,56,216


In [12]:
#Check zipcode lables
wazipcenters

,ZIP,LAT,LNG,cluster10KEY,cluster100KEY,cluster300KEY
32308,98001,47.310617,-122.263291,4,99,164
32309,98002,47.308286,-122.216812,4,22,164
32310,98003,47.304801,-122.316969,4,99,16
32311,98004,47.618337,-122.205341,4,51,210
32312,98005,47.614533,-122.168798,4,51,210
...,...,...,...,...,...,...
32900,99362,46.101083,-118.313609,8,33,65
32901,99363,46.060441,-118.883545,8,54,221
32902,99371,46.800013,-118.311494,8,9,126
32903,99401,46.087207,-117.249856,2,64,213


In [13]:
merged1=pd.merge(fatalcrash,wazipcenters, left_on='dzip', right_on='ZIP', how='left')

In [14]:
#full dataset with zipcodes, clusters, and driver zipcode lables
merged1

,year,case,par,repjur,crash_dt,crash_tm,accday,accmon,holiday,county,...,zip_code,cluster10,cluster100,cluster300,ZIP,LAT,LNG,cluster10KEY,cluster100KEY,cluster300KEY
0,2017,1,E628946,2.0,01/01/2017,2:12,1,1,1.0,61,...,98201.0,6,7,7,98204.0,47.901362,-122.260891,6.0,39.0,58.0
1,2017,2,E627989,26.0,01/02/2017,17:14,2,1,1.0,35,...,98359.0,4,28,129,98465.0,47.251761,-122.536065,4.0,41.0,55.0
2,2017,4,3747633,263.0,01/01/2017,18:47,1,1,1.0,33,...,98103.0,4,15,15,98125.0,47.716513,-122.295829,4.0,15.0,134.0
3,2017,5,E628691,4.0,01/01/2017,3:50,1,1,1.0,63,...,99219.0,2,66,72,99021.0,47.848267,-117.189612,2.0,59.0,227.0
4,2017,6,3746306,263.0,01/05/2017,9:53,5,1,0.0,33,...,98174.0,4,56,149,98065.0,47.584223,-121.791951,4.0,57.0,158.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4127,2021,636,Incid08,334.0,08/08/2021,9:38,8,8,NaN,47,...,99155.0,9,45,78,99155.0,48.249063,-118.965682,9.0,45.0,78.0
4128,2021,638,Incid09,334.0,09/05/2021,1:36,5,9,NaN,19,...,3971.0,1,1,1,99138.0,48.205843,-118.395823,9.0,12.0,145.0
4129,2021,639,EB48605,263.0,04/21/2021,17:32,21,4,NaN,33,...,98144.0,4,56,216,98503.0,47.022032,-122.797103,0.0,44.0,49.0
4130,2021,639,EB48605,263.0,04/21/2021,17:32,21,4,NaN,33,...,98144.0,4,56,216,98144.0,47.585627,-122.291960,4.0,56.0,216.0


In [15]:
# save updated dataframe to a new CSV file comparing 3 Ks and zipcodes
merged1.to_csv('Data_Washington_clusters.csv', index=False)